In [1]:
import cv2
import torch
import requests
import numpy as np
import pandas as pd
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image
import os

In [30]:
video_path = r'./training_set/Videos/'
frame_path = r'./training_set/Frames/'
n_frame_path = r'./training_set/N-Frames/'
feature_path = r'./training_set/Features/AlexNetFC7/'

In [22]:
error_log = list()

In [23]:
for filename in os.listdir(feature_path):
    video_name = filename[0:5] #because videos in Features use format xxxxx-xxx.csv
    frame_num = filename[6:9]
    frame_filename = frame_path + video_name + '-' + frame_num + '.png'
    if not os.path.exists(frame_filename):
        cap = cv2.VideoCapture(video_path + str(int(video_name)) + '.mp4')
        cap.set(cv2.CAP_PROP_POS_FRAMES, int(frame_num) - 1)
        res, frame = cap.read()
        if res == 0:
            error_log.append((video_name, frame_num))
        else:
            cv2.imwrite(frame_filename, frame)
        cap.release()

In [24]:
print(error_log)

[('00165', '186'), ('00261', '179'), ('02152', '172'), ('04461', '165'), ('05657', '129')]


In [6]:
# Manually iterate through error_log video frames until we get the frame

for video_name, frame_number in error_log:
    cap = cv2.VideoCapture(video_path + str(int(video_name)) + ".mp4")
    i = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if ret == False:
            print(f"Did not find frame {frame_number} of {video_name}")
            break
        if i == int(frame_number):
            cv2.imwrite(frame_path + video_name + '-' + frame_number + '.png', frame)
            break
        i += 1
    cap.release()

# Extract n frame per video

In [18]:
def get_total_frame(video):
    total = 0

    while True:
        (grabbed, frame) = video.read()
        
        if not grabbed:
            break
        
        total += 1
    
    return total

In [19]:
def split(a, n):
    k, m = divmod(len(a), n)
    return (a[i * k + min(i, m):(i + 1) * k + min(i + 1, m)] for i in range(n))

In [20]:
def get_frame_index(total_frame, n):
    list_of_index = list()
    list_of_chunk = list(split(range(1, total_frame+1), n))
    for i in range(n):
        list_of_index.append(list_of_chunk[i][0])
    return list_of_index

In [28]:
n = 8 #number of frame we want to extract
error_log_2 = list()


for filename in os.listdir(feature_path):
    video_name = filename[0:5]
    cap = cv2.VideoCapture(video_path + str(int(video_name)) + '.mp4')

    total_frame = get_total_frame(cap) #total number of frame of this video

    frame_index = get_frame_index(total_frame, n) #the index of the frame of interest

    for frame_num in frame_index:
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_num - 1)
        res, frame = cap.read()
        if res == 0:
            error_log_2.append((video_path, frame_num))
        else:
            cv2.imwrite(n_frame_path + video_name + '-' + '{:0>3}'.format(frame_num-1) + '.png', frame)

    cap.release()

In [29]:
print(error_log_2)

[]
